In [1]:
import os
import cv2
import sys
from zipfile import ZipFile
from urllib.request import urlretrieve


# ========================-Downloading Assets-========================
def download_and_unzip(url, save_path):
    print(f"Downloading and extracting assets....", end="")

    # Downloading zip file using urllib package.
    urlretrieve(url, save_path)

    try:
        # Extracting zip file using the zipfile package.
        with ZipFile(save_path) as z:
            # Extract ZIP file contents in the same directory.
            z.extractall(os.path.split(save_path)[0])

        print("Done")

    except Exception as e:
        print("\nInvalid file.", e)


URL = r"https://www.dropbox.com/s/efitgt363ada95a/opencv_bootcamp_assets_12.zip?dl=1"
asset_zip_path = os.path.join(os.getcwd(), f"opencv_bootcamp_assets_12.zip")

# Download if assets ZIP does not exist.
if not os.path.exists(asset_zip_path):
    download_and_unzip(URL, asset_zip_path)

# ========================End of Assets Downloading========================

# Ensure model files are present
if not os.path.exists('deploy.prototxt') or not os.path.exists('res10_300x300_ssd_iter_140000_fp16.caffemodel'):
    print("Required model files not found!")
    exit()

# Initialize video capture from webcam
source = cv2.VideoCapture(0)  # Change 0 for default webcam

if not source.isOpened():
    print("Error: Cannot access the camera")
    exit()

win_name = "Camera Preview"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)

# Load pre-trained face detection model
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000_fp16.caffemodel")

# Model parameters
in_width = 300
in_height = 300
mean = [104, 117, 123]
conf_threshold = 0.7

while cv2.waitKey(1) != 27:  # Press 'Esc' to exit
    has_frame, frame = source.read()
    if not has_frame:
        break
    frame = cv2.flip(frame, 1)

    # Create a 4D blob from the frame
    blob = cv2.dnn.blobFromImage(frame, 1.0, (in_width, in_height), mean, swapRB=False, crop=False)
    net.setInput(blob)
    detections = net.forward()

    # Loop through detections and draw bounding boxes
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x_top_left = int(detections[0, 0, i, 3] * frame.shape[1])
            y_top_left = int(detections[0, 0, i, 4] * frame.shape[0])
            x_bottom_right = int(detections[0, 0, i, 5] * frame.shape[1])
            y_bottom_right = int(detections[0, 0, i, 6] * frame.shape[0])

            cv2.rectangle(frame, (x_top_left, y_top_left), (x_bottom_right, y_bottom_right), (0, 255, 0), 2)
            label = "Confidence: %.4f" % confidence
            label_size, base_line = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(frame, (x_top_left, y_top_left - label_size[1]), (x_top_left + label_size[0], y_top_left + base_line), (255, 255, 255), cv2.FILLED)
            cv2.putText(frame, label, (x_top_left, y_top_left), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    # Show the frame with face detection
    t, _ = net.getPerfProfile()
    label = "Inference time: %.2f ms" % (t * 1000.0 / cv2.getTickFrequency())
    cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))
    cv2.imshow(win_name, frame)

# Release the capture and close the window
source.release()
cv2.destroyWindow(win_name)
